In [248]:
from keystonexl.client import PredatorClient
from keystonexl.base import PitchSide
import numpy as np
import enum
from math import isnan
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.8 ms


In [2]:
base_path = '/Users/jason.katz/Downloads'
predator_client = PredatorClient.from_paths(moment_path='{}/SAMPLE.hudlproj/momentsSave.json'.format(base_path), 
                                            tracking_path='{}/SAMPLE.hudlproj/trackingSave.json'.format(base_path))

time: 4.22 s


In [360]:
from bokeh.charts import Scatter
from bokeh.io import output_notebook, show
import pandas as pd
from bokeh.plotting import figure

time: 1.49 ms


In [361]:
output_notebook()

Loading BokehJS ...

time: 6.99 ms


In [679]:
for shot in games[0].shots:
    if not goalkeeper_in_shot_triangle(shot):
        shot_chart = figure(plot_width=105*7, plot_height=68*7)
        shot_chart.quad(top=[69], bottom=[-1], left=[-1], right=[106], color="green", alpha=.75)
        shot_chart.multi_line([[18, 18],[87, 87]], [[12, 56],[12, 56]], line_width = 3, color = 'white')
        shot_chart.line([0, 18], [12, 12], line_width = 3, color = 'white')
        shot_chart.line([87, 105], [12, 12], line_width = 3, color = 'white')
        shot_chart.line([0, 18], [56, 56], line_width = 3, color = 'white')
        shot_chart.line([87, 105], [56, 56], line_width = 3, color = 'white')
        shot_chart.line([6, 6], [24, 44], line_width = 3, color = 'white')
        shot_chart.line([99, 99], [24, 44], line_width = 3, color = 'white')
        shot_chart.line([0, 6], [24, 24], line_width = 3, color = 'white')
        shot_chart.line([99, 105], [24, 24], line_width = 3, color = 'white')
        shot_chart.line([0, 6], [44, 44], line_width = 3, color = 'white')
        shot_chart.line([99, 105], [44, 44], line_width = 3, color = 'white')
        shot_chart.line([52.5, 52.5], [0, 68], line_width = 3, color = 'white')
        shot_chart.line([0, 0], [0, 68], line_width = 3, color = 'white')
        shot_chart.line([105, 105], [0, 68], line_width = 3, color = 'white')
        shot_chart.line([0, 105], [0, 0], line_width = 3, color = 'white')
        shot_chart.line([0, 105], [68, 68], line_width = 3, color = 'white')
        shot_chart.line([-.5, .5], [38, 38], line_width = 4, color = 'black')
        shot_chart.line([-.5, .5], [30, 30], line_width = 4, color = 'black')
        shot_chart.line([104.5, 105.5], [30, 30], line_width = 4, color = 'black')
        shot_chart.line([104.5, 105.5], [38, 38], line_width = 4, color = 'black')
        shot_chart.circle(12, 34, color="white", size=10)
        shot_chart.circle(93, 34, color="white", size=10)
        shot_chart.circle(52.5, 34, color="white", size=10)
        shot_chart.arc(52.5, 34, radius=10, start_angle=0, end_angle=6.28, color="white", line_width=3)
        shot_chart.arc(12, 34, radius=10, start_angle=5.34, end_angle=.92, color="white", line_width=3)
        shot_chart.arc(93, 34, radius=10, start_angle=2.21, end_angle=4.08, color="white", line_width=3)
        if shot.x > 0:
            shot_chart.line([shot.x+52.5, 105, 105, shot.x+52.5], [shot.y+34, 30, 38, shot.y+34], color='orange', line_width=3)
        else:
            shot_chart.line([shot.x+52.5, 0, 0, shot.x+52.5], [shot.y+34, 30, 38, shot.y+34], color='orange', line_width=3)
        shot_chart.square(shot.x+52.5, shot.y+34, color='blue', size=11)
        shot_chart.square(shot.goalkeeper_x+52.5, shot.goalkeeper_y+34, color='red', size=11)
        shot_chart.x_range.start, shot_chart.x_range.end, shot_chart.y_range.start, shot_chart.y_range.end = [-1, 106, -1, 69]
        print(shot.player_coords)
        print(shot.goalkeeper_coords)
        print(shot.shot_triangle_area)
        print(shot.made.name)
        show(shot_chart)

[ 50.24000168  -0.56999999]
[ 51.5         -2.94000006]
9.83999328613
no


[-42.09000015 -11.81000042]
[-48.95999908  -0.64999998]
42.4399993896
no


[-41.02000046   2.91000009]
[-51.68999863  -3.8499999 ]
46.7199981689
no


[-50.93999863 -25.04999924]
[-51.47999954  -3.27999997]
7.04000549316
no


[-37.70999908   3.51999998]
[-51.08000183  -3.55999994]
59.9600036621
no


time: 20.4 s


In [3]:
class AlwaysTrue(enum.Enum):
    yes = 1
    no = 0

class FreeKick(enum.Enum):
    yes = 1
    no = 0
    
class Corner(enum.Enum):
    yes = 1
    no = 0
    
class Cross(enum.Enum):
    yes = 1
    no = 0
    
class ThrowIn(enum.Enum):
    yes = 1
    no = 0
    
class Header(enum.Enum):
    yes = 1
    no = 0
    
class Made(enum.Enum):
    yes = 1
    no = 0

time: 10.4 ms


In [497]:
test = Game(predator_client)

time: 1.27 s


In [582]:
test2 = Game(pc)

time: 1.51 s


In [538]:
class Game(object):
    def __init__(self, predator_client):
        self.predator_client = predator_client
        self.game_id = np.random.rand()
        self.shots = self.extract_data()
        self.first_shot = self.shots[0]   
        self.first_shot_period_num = self.first_shot['period_num']
        self.first_shot_team_id = self.predator_client.get_value(self.first_shot, 'team_id')
        self.first_shot_team_type = self.predator_client.team_type(self.first_shot_team_id)
        self.first_shot_goal_side = self.predator_client.estimate_defending_half(self.first_shot_team_type, 
                                                                            self.first_shot_period_num)
        self.goalkeeper_home_idx, self.goalkeeper_away_idx = self.get_goalkeepers()
        self.player_idxs = self.get_all_player_idxs()
        self.shots = [Shot(shot, self.game_id, self.predator_client, self.first_shot_period_num, self.first_shot_team_type, 
                           self.first_shot_goal_side, self.goalkeeper_home_idx, self.goalkeeper_away_idx, self.player_idxs) for shot in self.shots]
        
    @staticmethod    
    def is_own_goal(moment):
        own_goal = False
        for tag in moment['tags']:
            if tag['key'] == 'qualifier':
                if tag['value'] == 'Own goal':
                    own_goal = True
        return own_goal

    def extract_data(self):
        extracted_moments = self.predator_client.moments
        shots = []
        for idx, moment in enumerate(extracted_moments):
            extracted_moments[idx]['index'] = idx
            frame = self.predator_client.frame_from_time(moment['period_num'], moment['end_ms'])
            try:
                player_id = self.predator_client.get_value(moment, 'player_id')
                player_idx = self.predator_client.lookup_from_df('queryable_id', player_id, 'tracking_index')
                extracted_moments[idx]['x'] = self.predator_client.tracking_array[frame, player_idx, :2][0]
            except:
                if moment['end_ms'] < 1.0:
                    extracted_moments[idx]['x'] = 0.0
                else:
                    extracted_moments[idx]['x'] = extracted_moments[idx-1]['x']
            if self.predator_client.has_tag(moment, 'shot') and not self.is_own_goal(moment):
                moment['speed_3'] = abs(extracted_moments[idx-3]['x'] - moment['x']) / (moment['end_ms'] - extracted_moments[idx-3]['end_ms']) * 1000
                moment['speed_5'] = abs(extracted_moments[idx-5]['x'] - moment['x']) / (moment['end_ms'] - extracted_moments[idx-5]['end_ms']) * 1000
                shots.append(moment)
        return shots
    
    def get_goalkeepers(self):
        player_ids = [column[11:] for column in self.predator_client.moment_tag_df.columns if 'player_id__p_' in column]
        
        players = [self.predator_client.lookup_from_df('queryable_id', player, 'tracking_index') for player in player_ids]
        all_goalkeepers = [player for player in self.predator_client.players if player['player_position'] == 'Goalkeeper']
        goalkeepers = [goalkeeper for idx, goalkeeper in enumerate(all_goalkeepers) if goalkeeper['hudl_athlete_id'] in player_ids]
        for goalkeeper in goalkeepers:
            goalkeeper_id = goalkeeper['hudl_athlete_id']
            if goalkeeper['player_team']['team_type'] == 'home':
                goalkeeper_home_idx = self.predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')
            else:
                goalkeeper_away_idx = self.predator_client.lookup_from_df('queryable_id', goalkeeper_id, 'tracking_index')
        return goalkeeper_home_idx, goalkeeper_away_idx
    
    def get_all_player_idxs(self):
        player_idxs = {'home': [], 'away': []}
        for player in self.predator_client.players:
            player_idx = self.predator_client.lookup_from_df('queryable_id', player['hudl_athlete_id'], 'tracking_index')
            if player_idx != -1:
                team = player['player_team']['team_type']
                player_idxs[team].append(player_idx)
        return player_idxs

time: 75 ms


In [553]:
class Shot(object):
    def __init__(self, shot, game_id, predator_client, first_shot_period_num, first_shot_team_type, first_shot_goal_side,
                goalkeeper_home_idx, goalkeeper_away_idx, player_idxs):
        self.shot = shot
        self.game_id = game_id
        self.predator_client = predator_client
        self.first_shot_period_num = first_shot_period_num
        self.first_shot_team_type = first_shot_team_type
        self.first_shot_goal_side = first_shot_goal_side
        self.goalkeeper_home_idx = goalkeeper_home_idx
        self.goalkeeper_away_idx = goalkeeper_away_idx
        self.player_idxs = player_idxs
    
            
    @property
    def always_true(self):
        return AlwaysTrue.yes
    
    @property
    def pitch_width(self):
        return self.predator_client.pitch_width
    
    @property
    def pitch_height(self):
        return self.predator_client.pitch_height
    
    @property
    def teams(self):
        return self.predator_client.teams
    
    def get_team_idx(self, team_type):
        for idx, team in enumerate(self.teams):
            if team['team_type'] == team_type:
                return idx
    
    @property
    def home_team_idx(self):
        return self.get_team_idx('home')
    
    @property
    def away_team_idx(self):
        return self.get_team_idx('away')
    
    @property
    def frame(self):
        return self.predator_client.frame_from_time(self.shot['period_num'], self.shot['end_ms'])
    
    @property
    def period_num(self):
        return self.shot['period_num']
    
    @property
    def team_id(self):
        return self.predator_client.get_value(self.shot, 'team_id')
    
    @property
    def home_team_id(self):
        return self.teams[self.home_team_idx]['hudl_team_id']
    
    @property
    def home_team_name(self):
        return self.teams[self.home_team_idx]['team_name']
    
    @property
    def away_team_id(self):
        return self.teams[self.away_team_idx]['hudl_team_id']
    
    @property
    def away_team_name(self):
        return self.teams[self.away_team_idx]['team_name']
        
    @property
    def team_type(self):
        return self.predator_client.team_type(self.team_id)
    
    @property
    def opposing_team_type(self):
        if self.team_type == 'home':
            return 'away'
        else:
            return 'home'
    
    @property
    def goal_side(self):
        if self.period_num == self.first_shot_period_num: 
            if self.team_type == self.first_shot_team_type:
                return self.first_shot_goal_side
            else:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
        elif self.period_num == 1 or self.period_num == 2:
            if self.team_type == self.first_shot_team_type:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
            else:
                return self.first_shot_goal_side
        else:
            return self.predator_client.estimate_defending_half(self.team_type, self.period_num )
        
    @property
    def goal_x(self):
        if self.goal_side == PitchSide.right:
            return self.predator_client.pitch_width * -.5
        else:
            return self.predator_client.pitch_width * .5
        
    @property
    def goal_coords(self):
        return np.array([self.goal_x, 0])
    
    @property
    def player_id(self):
        return self.predator_client.get_value(self.shot, 'player_id')
        
    @property
    def player_idx(self):
        return self.predator_client.lookup_from_df('queryable_id', self.player_id, 'tracking_index')
    
    @property
    def player_coords(self):
        return self.predator_client.tracking_array[self.frame, self.player_idx, :2]
    
    @property
    def goalkeeper_coords(self):
        if self.team_type == 'home':
            return self.predator_client.tracking_array[self.frame, self.goalkeeper_away_idx, :2]
        else:
            return self.predator_client.tracking_array[self.frame, self.goalkeeper_home_idx, :2]
        
    @property
    def opposing_players_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.opposing_team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def teamates_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def goalkeeper_x(self):
        return self.goalkeeper_coords[0]
    
    @property
    def goalkeeper_y(self):
        return self.goalkeeper_coords[1]
    
    @property
    def x(self):
        return self.player_coords[0]
    
    @property
    def y(self):
        return self.player_coords[1]
        
    @property
    def distance(self):
        return np.linalg.norm(self.player_coords - self.goal_coords)
    
    @property
    def angle(self):
        return np.rad2deg(np.arctan((abs(self.y)) / (self.pitch_width / 2 - abs(self.x))))
    
    def has_attribute(self, event_type):
        in_list = False
        while not in_list:
            for descriptor in self.shot['tags']:
                if descriptor['key'] == event_type:
                    in_list = True
            break
        return in_list
    
    @property
    def free_kick(self):
        if self.has_attribute('free kick'):
            return FreeKick.yes
        else:
            return FreeKick.no
        
    @property
    def corner(self):
        if self.has_attribute('corner'):
            return Corner.yes
        else:
            return Corner.no
        
    @property
    def cross(self):
        if self.has_attribute('cross'):
            return Cross.yes
        else:
            return Cross.no
        
    @property
    def throw_in(self):
        if self.has_attribute('throw in'):
            return ThrowIn.yes
        else:
            return ThrowIn.no
        
    @property
    def header(self):
        if self.has_attribute('header'):
            return Header.yes
        else:
            return Header.no
        
    @property
    def made(self):
        if self.has_attribute('goal'):
            return Made.yes
        else:
            return Made.no
        
    @property
    def speed_3(self):
        return self.shot['speed_3']
    
    @property
    def speed_5(self):
        return self.shot['speed_5']
    
    def inside_shot_triangle(self, defender_coords):
        defender_x = abs(defender_coords[0])
        defender_y = defender_coords[1]
        shot_x = abs(self.x)
        shot_y = self.y
        goal_post_x = self.pitch_width / 2
        slope_high = (shot_y - 4) / (shot_x - goal_post_x)
        slope_low = (shot_y + 4) / (shot_x - goal_post_x)
        max_y = shot_y + slope_high*(defender_x - shot_x)
        min_y = shot_y + slope_low*(defender_x - shot_x)
        return defender_y > min_y and defender_y < max_y and defender_x > shot_x and defender_x < goal_post_x
    
    @property
    def goalkeeper_in_shot_triangle(self):
        return self.inside_shot_triangle(self.goalkeeper_coords)
        
    @property
    def defenders_in_shot_triangle(self):
        num_defenders = 0
        for defender_coords in self.opposing_players_coords:
            num_defenders += self.inside_shot_triangle(defender_coords)
        return num_defenders
    
    @property
    def shot_triangle_area(self):
        base = 8
        height = self.pitch_width / 2 - abs(self.x)
        return .5 * base * height

time: 406 ms


In [400]:
import sys

time: 726 µs


In [401]:
sys.path.append('/Users/jason.katz/Downloads/predator-master/hudl_predator/analysis_tools')

time: 790 µs


In [403]:
import catalog_download_tool

time: 15.1 ms


In [589]:
import os
import zipfile
import shutil

time: 1 ms


In [412]:
tool = catalog_download_tool.CatalogDownloadTool()

2017-07-20 16:24:33,099 [WARNING] [ConfigurableValue] [25203] {"message": "Initializing configurable value for key=predator.analysis.aws_access_key with value=None"}
2017-07-20 16:24:33,100 [WARNING] [ConfigurableValue] [25203] {"message": "Initializing configurable value for key=predator.analysis.aws_secret_access_key with value=None"}
2017-07-20 16:24:33,158 [INFO] [CatalogDownloadTool] [25203] {"message": "building catalog manifest table"}
2017-07-20 16:25:23,337 [INFO] [CatalogDownloadTool] [25203] {"message": "catalog manifest table built", "entries": 448, "completion_time_s": 50.17815923690796}
2017-07-20 16:25:23,338 [INFO] [CatalogDownloadTool] [25203] {"message": "building catalog manifest cache"}
time: 50.3 s


In [675]:
games = []
for i in range(2):
    base_path = '/Users/jason.katz/Downloads/Data'
    tool.download_to_folder(tool.manifest_infos[i:i+1], base_path, dry_run=False, original_file_name_regex='')
    sub_folder = os.listdir(base_path)[1]
    base_path = base_path + '/' + sub_folder + '/'
    have_opta = False
    have_tracab = False
    have_f24_file = False
    have_srml_file = False
    have_xml_file = False
    have_dat_file = False
    for item in os.listdir(base_path):
        if '.zip' in item and ('opta' in item or 'Opta' in item):
            zip_ref = zipfile.ZipFile(base_path + item, 'r') # create zipfile object
            zip_ref.extractall(base_path) # extract file to dir
            zip_ref.close() # close file
            os.remove(base_path + item)
            folder_items = os.listdir(base_path)
            possible_folders = ['Opta', 'opta_import']
            for file_name in folder_items:
                if possible_folders[0] in file_name:
                    for file in os.listdir(base_path + file_name):
                        if file[0:3] == 'f24':
                            f24_file = file
                            have_f24_file = True
                        elif file[0:4] == 'srml':
                            srml_file = file
                            have_srml_file = True
                    break
                elif possible_folders[1] == file_name:
                    for file in os.listdir(base_path + file_name):
                        if file[0:3] == 'f24':
                            f24_file = file
                            have_f24_file = True
                        elif file[0:4] == 'srml':
                            srml_file = file  
                            have_srml_file = True
                    break
                if file_name[0:3] == 'f24':
                    f24_file = file_name
                    have_f24_file = True
                if file_name[0:4] == 'srml':
                    srml_file = file_name
                    have_srml_file = True
            if have_f24_file and have_srml_file:
                opta_source = OptaSource(
                    base_path + 'Opta/' + f24_file,
                    base_path + 'Opta/' + srml_file
                )
                opta_source.write(base_path + 'moments.json')
                have_opta = True
        elif '.zip' in item and 'tracab' in item:
            zip_ref = zipfile.ZipFile(base_path + item, 'r') # create zipfile object
            zip_ref.extractall(base_path) # extract file to dir
            zip_ref.close() # close file
            os.remove(base_path + item)
            for item in os.listdir(base_path):
                if item.endswith("tracabRaw"):
                    tracab_folder = item
            for item in os.listdir(base_path + tracab_folder):
                if item.endswith('.xml'):
                    xml_file = item
                    have_xml_file = True
                elif item.endswith('.dat'):
                    dat_file = item
                    have_dat_file = True
            if have_xml_file and have_dat_file:
                tracab_source = TracabSource(
                    base_path + tracab_folder + '/' + xml_file,
                    base_path + tracab_folder + '/' + dat_file
                )
                tracab_source.write(base_path + 'tracking.json')
                have_tracab = True
    if have_opta and have_tracab:
        pc = PredatorClient.from_paths('{}tracking.json'.format(base_path), 
                                   '{}moments.json'.format(base_path))
        games.append(Game(pc))
    else:
        print('Did not have both Opta and Tracab data to create a predator client for game {}'.format(i))
    shutil.rmtree(base_path)

2017-07-21 12:06:15,626 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading 6 Files to /Users/jason.katz/Downloads/Data", "n_files": 6, "size_GB": 0.468896002}
2017-07-21 12:06:15,627 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading predator_breakdown.xml", "dry_run": false}
2017-07-21 12:06:17,849 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading sync_state_803532.json", "dry_run": false}
2017-07-21 12:06:18,160 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading manifest.json", "dry_run": false}
2017-07-21 12:06:18,482 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading full_event_803532.hdf5", "dry_run": false}
2017-07-21 12:06:45,349 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading opta_import.zip", "dry_run": false}
2017-07-21 12:06:45,648 [INFO] [CatalogDownloadTool] [25203] {"message": "Downloading tracab_import.zip", "dry_run": false}
2017-07-21 12:06:53,824 [WARNING] [keystonexl.moments.opta] [25203] {"message

In [418]:
from keystonexl.client import PredatorClient
from keystonexl.moments.opta import OptaSource
from keystonexl.tracking.tracab import TracabSource

time: 1.36 ms


In [541]:
base_path = '/Users/jason.katz/Downloads/Data/5797b6ae2357b70ecc0bde1f/'

time: 1.28 ms


In [542]:
opta_source = OptaSource(
    '{}Opta/f24-8-2015-803384-eventdetails.xml'.format(base_path),
    '{}Opta/srml-8-2015-f803384-matchresults.xml'.format(base_path)
)

2017-07-20 18:16:57,996 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,001 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,032 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,044 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,045 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,074 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,106 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,110 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing Opta event: type_id=74"}
2017-07-20 18:16:58,139 [WARNING] [keystonexl.moments.opta] [25203] {"message": "Missing

In [543]:
opta_source.write('{}/moments.json'.format(base_path))

time: 52.4 ms


In [544]:
tracab_source = TracabSource(
    '{}803384_tracabRaw/803384_Arsenal-Chelsea_metadata.xml'.format(base_path),
    '{}803384_tracabRaw/803384_Arsenal-Chelsea.dat'.format(base_path)
)

time: 40 s


In [545]:
tracab_source.write('{}/tracking.json'.format(base_path))

time: 13.3 s


In [546]:
pc = PredatorClient.from_paths('{}/tracking.json'.format(base_path),
                               '{}/moments.json'.format(base_path))

time: 4.45 s


In [ ]:
class ShotStatic(object):
    def __init__(self, shot, game_id, predator_client, first_shot_period_num, first_shot_team_type, first_shot_goal_side,
                goalkeeper_home_idx, goalkeeper_away_idx, player_idxs):
        self.shot = shot
        self.game_id = game_id
        self.predator_client = predator_client
        self.first_shot_period_num = first_shot_period_num
        self.first_shot_team_type = first_shot_team_type
        self.first_shot_goal_side = first_shot_goal_side
        self.goalkeeper_home_idx = goalkeeper_home_idx
        self.goalkeeper_away_idx = goalkeeper_away_idx
        self.player_idxs = player_idxs
    
            
    @property
    def always_true(self):
        return AlwaysTrue.yes
    
    @property
    def pitch_width(self):
        return self.predator_client.pitch_width
    
    @property
    def pitch_height(self):
        return self.predator_client.pitch_height
    
    @property
    def teams(self):
        return self.predator_client.teams
    
    def get_team_idx(self, team_type):
        for idx, team in enumerate(self.teams):
            if team['team_type'] == team_type:
                return idx
    
    @property
    def home_team_idx(self):
        return self.get_team_idx('home')
    
    @property
    def away_team_idx(self):
        return self.get_team_idx('away')
    
    @property
    def frame(self):
        return self.predator_client.frame_from_time(self.shot['period_num'], self.shot['end_ms'])
    
    @property
    def period_num(self):
        return self.shot['period_num']
    
    @property
    def team_id(self):
        return self.predator_client.get_value(self.shot, 'team_id')
    
    @property
    def home_team_id(self):
        return self.teams[self.home_team_idx]['hudl_team_id']
    
    @property
    def home_team_name(self):
        return self.teams[self.home_team_idx]['team_name']
    
    @property
    def away_team_id(self):
        return self.teams[self.away_team_idx]['hudl_team_id']
    
    @property
    def away_team_name(self):
        return self.teams[self.away_team_idx]['team_name']
        
    @property
    def team_type(self):
        return self.predator_client.team_type(self.team_id)
    
    @property
    def opposing_team_type(self):
        if self.team_type == 'home':
            return 'away'
        else:
            return 'home'
    
    @property
    def goal_side(self):
        if self.period_num == self.first_shot_period_num: 
            if self.team_type == self.first_shot_team_type:
                return self.first_shot_goal_side
            else:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
        elif self.period_num == 1 or self.period_num == 2:
            if self.team_type == self.first_shot_team_type:
                if self.first_shot_goal_side.name == 'left':
                    return PitchSide.right
                else:
                    return PitchSide.left
            else:
                return self.first_shot_goal_side
        else:
            return self.predator_client.estimate_defending_half(self.team_type, self.period_num )
        
    @property
    def goal_x(self):
        if self.goal_side == PitchSide.right:
            return self.predator_client.pitch_width * -.5
        else:
            return self.predator_client.pitch_width * .5
        
    @property
    def goal_coords(self):
        return np.array([self.goal_x, 0])
    
    @property
    def player_id(self):
        return self.predator_client.get_value(self.shot, 'player_id')
        
    @property
    def player_idx(self):
        return self.predator_client.lookup_from_df('queryable_id', self.player_id, 'tracking_index')
    
    @property
    def player_coords(self):
        return self.predator_client.tracking_array[self.frame, self.player_idx, :2]
    
    @property
    def goalkeeper_coords(self):
        if self.team_type == 'home':
            return self.predator_client.tracking_array[self.frame, self.goalkeeper_away_idx, :2]
        else:
            return self.predator_client.tracking_array[self.frame, self.goalkeeper_home_idx, :2]
        
    @property
    def opposing_players_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.opposing_team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def teamates_coords(self):
        return [self.predator_client.tracking_array[self.frame, player, :2] for player in self.player_idxs[self.team_type] if not isnan(self.predator_client.tracking_array[self.frame, player, 1])]
    
    @property
    def goalkeeper_x(self):
        return self.goalkeeper_coords[0]
    
    @property
    def goalkeeper_y(self):
        return self.goalkeeper_coords[1]
    
    @property
    def x(self):
        return self.player_coords[0]
    
    @property
    def y(self):
        return self.player_coords[1]
        
    @property
    def distance(self):
        return np.linalg.norm(self.player_coords - self.goal_coords)
    
    @property
    def angle(self):
        return np.rad2deg(np.arctan((abs(self.y)) / (self.pitch_width / 2 - abs(self.x))))
    
    def has_attribute(self, event_type):
        in_list = False
        while not in_list:
            for descriptor in self.shot['tags']:
                if descriptor['key'] == event_type:
                    in_list = True
            break
        return in_list
    
    @property
    def free_kick(self):
        if self.has_attribute('free kick'):
            return FreeKick.yes
        else:
            return FreeKick.no
        
    @property
    def corner(self):
        if self.has_attribute('corner'):
            return Corner.yes
        else:
            return Corner.no
        
    @property
    def cross(self):
        if self.has_attribute('cross'):
            return Cross.yes
        else:
            return Cross.no
        
    @property
    def throw_in(self):
        if self.has_attribute('throw in'):
            return ThrowIn.yes
        else:
            return ThrowIn.no
        
    @property
    def header(self):
        if self.has_attribute('header'):
            return Header.yes
        else:
            return Header.no
        
    @property
    def made(self):
        if self.has_attribute('goal'):
            return Made.yes
        else:
            return Made.no
        
    @property
    def speed_3(self):
        return self.shot['speed_3']
    
    @property
    def speed_5(self):
        return self.shot['speed_5']
    
    def inside_shot_triangle(self, defender_coords):
        defender_x = abs(defender_coords[0])
        defender_y = defender_coords[1]
        shot_x = abs(self.x)
        shot_y = self.y
        goal_post_x = self.pitch_width / 2
        slope_high = (shot_y - 4) / (shot_x - goal_post_x)
        slope_low = (shot_y + 4) / (shot_x - goal_post_x)
        max_y = shot_y + slope_high*(defender_x - shot_x)
        min_y = shot_y + slope_low*(defender_x - shot_x)
        return defender_y > min_y and defender_y < max_y and defender_x > shot_x and defender_x < goal_post_x
    
    @property
    def goalkeeper_in_shot_triangle(self):
        return self.inside_shot_triangle(self.goalkeeper_coords)
        
    @property
    def defenders_in_shot_triangle(self):
        num_defenders = 0
        for defender_coords in self.opposing_players_coords:
            num_defenders += self.inside_shot_triangle(defender_coords)
        return num_defenders
    
    @property
    def shot_triangle_area(self):
        base = 8
        height = self.pitch_width / 2 - abs(self.x)
        return .5 * base * height